In [8]:
import os, sys

def find_project_root(start_path):
    cur = os.path.abspath(start_path)
    while True:
        if os.path.isdir(os.path.join(cur, "src")):
            return cur
        parent = os.path.dirname(cur)
        if parent == cur:
            break
        cur = parent
    raise RuntimeError("Project root not found.")

ROOT = find_project_root(os.getcwd())
sys.path.append(ROOT)
os.chdir(ROOT)

print("ROOT:", ROOT)
print("CWD :", os.getcwd())


ROOT: c:\Users\sam\Documents\VSCode_tunnel\cv_2024_upscale
CWD : c:\Users\sam\Documents\VSCode_tunnel\cv_2024_upscale


In [9]:
from src.data.dataset_pairs import UpscaleDataset
from torch.utils.data import DataLoader

dataset = UpscaleDataset(lr_dir='data/train_lr', hr_dir='data/train_hr')    
dl = DataLoader(dataset, batch_size=4, shuffle=True)
lr, hr = next(iter(dl))
print(lr.shape, hr.shape)


Found 492 image pairs.
torch.Size([4, 3, 64, 64]) torch.Size([4, 3, 256, 256])


In [10]:
import torch.nn as nn

m = nn.Conv2d(3, 3, kernel_size=3, padding=1)

lr, hr = dataset[0]
lr = lr.unsqueeze(0)   # (1,3,64,64)

out = m(lr)
print(out.shape)


torch.Size([1, 3, 64, 64])


In [11]:
import torch
import torch.nn as nn

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(m.parameters(), lr=1e-4)

lr, hr = dataset[0]
lr = lr.unsqueeze(0)
hr = hr.unsqueeze(0)

out = m(lr)
loss = criterion(out, hr[:, :, ::4, ::4])
loss.backward()

print(loss.item())


0.6240766644477844
